In [6]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [7]:
# !pip install stable-baselines3[extra]
# !pip install gym[all]



In [2]:
import os
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [4]:
environment_name = "CartPole-v1"
env = gym.make(environment_name, render_mode='human')

episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, trunc, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:20.0
Episode:2 Score:15.0
Episode:3 Score:14.0
Episode:4 Score:37.0
Episode:5 Score:13.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [5]:
env.action_space

Discrete(2)

In [6]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

0

In [7]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([-2.4341729e+00,  1.8044755e+38, -7.9495184e-02,  1.6584086e+38],
      dtype=float32)

# 3. Train an RL Model

In [8]:
log_path = os.path.join('Training', 'Logs')

In [9]:
log_path

'Training\\Logs'

In [10]:
# import torch
# torch.__version__

In [11]:
 env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)

Using cpu device


C:\Users\Jasmi\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [17]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_6
-----------------------------
| time/              |      |
|    fps             | 1580 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1109        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008313846 |
|    clip_fraction        | 0.11        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.000833   |
|    learning_rate        | 0.0003      |
|    loss                 | 5.23        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0171     |
|    value_loss           | 48.3        |
-----------------------------------------
---

# 4. Save and Reload Model

In [12]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [13]:
model.save(PPO_Path)

In [14]:
del model

In [15]:
model = PPO.load(PPO_Path, env=env)

# 4. Evaluation

In [16]:
from stable_baselines3.common.evaluation import evaluate_policy

In [17]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\Jasmi\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
C:\Users\Jasmi\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:234: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(41.0, 8.977750274985375)

In [18]:
env.close()

# 5. Test Model

In [19]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

info [{'TimeLimit.truncated': False, 'terminal_observation': array([-0.13734373, -1.0082302 ,  0.21179585,  1.6151781 ], dtype=float32)}]


In [1]:
environment_name = "CartPole-v1"
env = gym.make(environment_name, render_mode='human')

episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset() # state to obs
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = model.predict(obs)
        n_state, reward, done, trunc, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

NameError: name 'gym' is not defined

# 6. Viewing Logs in Tensorboard

In [135]:
training_log_path = os.path.join(log_path, 'PPO_3')

In [136]:
# !tensorboard --logdir={training_log_path}

# 7. Adding a callback to the training Stage

In [137]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [138]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [139]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [140]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [141]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [142]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_5
-----------------------------
| time/              |      |
|    fps             | 1695 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1152        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007393534 |
|    clip_fraction        | 0.0835      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | 0.00264     |
|    learning_rate        | 0.0003      |
|    loss                 | 7.49        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0105     |
|    value_loss           | 51.1        |
-----------------------------------------
---

In [143]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [144]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(388.4, 141.1638763990278)

In [145]:
env.close()

# 8. Changing Policies

In [146]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [147]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cpu device


C:\Users\Jasmi\anaconda3\lib\site-packages\stable_baselines3\common\policies.py:460: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [148]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 1104 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 712          |
|    iterations           | 2            |
|    time_elapsed         | 5            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0143183125 |
|    clip_fraction        | 0.184        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.683       |
|    explained_variance   | 0.00927      |
|    learning_rate        | 0.0003       |
|    loss                 | 3.48         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.021       |
|    value_loss           | 22.9         |
------------------------------------------
----------------

# 9. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()